# Pydantic AI Walkthrough

Inspired bij: https://ai.pydantic.dev/agents/

Setup the environment

In [10]:
import dotenv
import os
import logfire

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")
logfire_token = os.getenv("LOGFIRE_TOKEN")

# configure logfire
logfire.configure(token=logfire_token)
logfire.instrument_pydantic_ai()

1 Definieer een basis agent

In [11]:
from pydantic_ai import Agent
from pydantic_ai.models.anthropic import AnthropicModel

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = AnthropicModel(os.getenv("DEFAULT_MODEL"))

basic_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

Stel een vraag aan de agent

In [12]:
result = basic_agent.run_sync("geef een defintie van een agent")
print(result.output)

05:40:19.286 basic_agent run
05:40:19.286   chat claude-3-haiku-20240307


Logfire project URL: ]8;id=330515;https://logfire-eu.pydantic.dev/elbrink/claude-test\https://logfire-eu.pydantic.dev/elbrink/claude-test]8;;\

Een agent is in de informatica een softwareprogramma of computersysteem dat autonoom en adaptief handelt om een bepaald doel te bereiken. Enkele kenmerken van een agent zijn:

1. Autonomie: De agent kan zelfstandig beslissingen nemen en acties uitvoeren zonder directe menselijke interventie.

2. Proactief gedrag: De agent kan zelf initiatieven nemen om doelen te bereiken, in plaats van alleen te reageren op externe prikkels.

3. Waarneming van de omgeving: De agent kan informatie opnemen uit zijn omgeving, zoals sensoren, gebruikersinvoer of andere gegevenbronnen.

4. Adaptief vermogen: De agent kan zijn gedrag aanpassen aan veranderende omstandigheden of nieuwe informatie, om zijn doelen te blijven bereiken.

5. Communicatie: Agents kunnen onderling communiceren en samenwerken om complexere taken uit te voeren.

Agenten worden ingezet in uiteenlopende toepassingen, zoals kunstmatige intelligentie, robotica, informatiesystemen en beslissingsondersteuning. De mate van autonomie en de sp

Definieer tools voor de agent

In [13]:
tool_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)


@tool_agent.tool_plain
def tool_bereken_btw(bedrag: float, btw_percentage: float):
    """
    Bereken BTW over een bedrag.
    """
    return tf.calculate_btw(bedrag, btw_percentage)


@tool_agent.tool_plain
def tool_bereken_korting(bedrag: float, korting_percentage: float):
    """
    Bereken korting over een bedrag.
    """
    return tf.calculate_discount(bedrag, korting_percentage)


@tool_agent.tool_plain
def tool_get_inwoners_gemeente(gemeente_naam: str, jaar: str):
    """
    Haal inwonersaantallen op van Nederlandse gemeenten via CBS API.
    """
    return tf.get_inwoners_gemeente(gemeente_naam, jaar)

Test een aantal tool calls

In [14]:
result2 = tool_agent.run_sync(
    "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld"
)
print(result2.output)

05:40:22.115 tool_agent run
05:40:22.116   chat claude-3-haiku-20240307
05:40:23.653   running 1 tool
05:40:23.654     running tool: tool_bereken_btw
05:40:23.655   chat claude-3-haiku-20240307
05:40:25.563   running 1 tool
05:40:25.563     running tool: tool_bereken_korting
05:40:25.565   chat claude-3-haiku-20240307
Met 15% korting op het bedrag inclusief BTW van 121 euro, kom je uit op een eindprijs van 102,85 euro.

Samengevat:
- Oorspronkelijke prijs excl. BTW: 100 euro
- BTW bedrag: 21 euro
- Prijs incl. BTW: 121 euro
- Korting van 15%: 18,15 euro
- Uiteindelijke prijs: 102,85 euro


In [15]:
result = tool_agent.run_sync(
    "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024. Laat ook het aantal inwoners van Groningen zien."
)
print(result.output)

05:40:27.056 tool_agent run
05:40:27.057   chat claude-3-haiku-20240307
05:40:28.387   running 1 tool
05:40:28.388     running tool: tool_get_inwoners_gemeente
05:40:29.062   chat claude-3-haiku-20240307
05:40:30.164   running 1 tool
05:40:30.165     running tool: tool_get_inwoners_gemeente
05:40:30.716   chat claude-3-haiku-20240307
05:40:31.745   running 1 tool
05:40:31.745     running tool: tool_get_inwoners_gemeente
05:40:32.165   chat claude-3-haiku-20240307
05:40:33.235   running 1 tool
05:40:33.236     running tool: tool_get_inwoners_gemeente
05:40:33.643   chat claude-3-haiku-20240307
05:40:34.806   running 1 tool
05:40:34.807     running tool: tool_get_inwoners_gemeente
05:40:35.213   chat claude-3-haiku-20240307
05:40:36.324   running 1 tool
05:40:36.324     running tool: tool_get_inwoners_gemeente
05:40:36.728   chat claude-3-haiku-20240307
05:40:37.827   running 1 tool
05:40:37.828     running tool: tool_get_inwoners_gemeente
05:40:38.236   chat claude-3-haiku-20240307
Op b

In [16]:
result = tool_agent.run_sync(
    "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan."
)
print(result.output)

05:40:40.057 tool_agent run
05:40:40.058   chat claude-3-haiku-20240307
05:40:42.006   running 1 tool
05:40:42.006     running tool: tool_get_inwoners_gemeente
05:40:42.591   chat claude-3-haiku-20240307
Volgens de CBS-data zal Eindhoven in 2025 ongeveer 249.054 inwoners hebben.

Als elk van deze inwoners 15 euro betaalt, dan zou dat het volgende opleveren:

Aantal inwoners: 249.054
Bedrag per inwoner: 15 euro
Totaal bedrag: 249.054 * 15 = 3.735.810 euro

Dus als alle inwoners van Eindhoven in 2025 elk 15 euro betalen, zou dat in totaal 3.735.810 euro opleveren.


Voeg geheugen toe aan de agent

In [17]:
mem_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

result = mem_agent.run_sync(
    "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor.",
    message_history=result2.all_messages(),
)
print(result.output)

05:40:44.287 mem_agent run
05:40:44.288   chat claude-3-haiku-20240307
Uit de vorige stappen kunnen we het volgende afleiden:

BTW bedrag:

